In [1]:
# ─── PARSE INPUT ────────────────────────────────────────────────────────────────

import re

with open('input16') as file:
    lines = file.read()

lines_ranges, lines_your_ticket, lines_nearby_tickets = lines.split('\n\n')

# ─── RANGES 
field_ranges = []
for string in lines_ranges.splitlines():
    name, numbers = string.split(':')
    field_ranges.append(list(map(int, re.findall('\d+', numbers))))

# ─── MY TICKET 
your_ticket = list(map(int, re.findall('\d+', lines_your_ticket)))

# ─── NEARBY TICKETS 
nearby_tickets = []
for line in lines_nearby_tickets.splitlines():
    match = re.findall('\d+', line)
    if match:
        nearby_tickets.append(list(map(int, match)))

In [2]:
# ─── CHALLENGE 1 ────────────────────────────────────────────────────────────────

def number_in_any_range(number):
    # check, if number is within one or more ranges
    number_valid = False
    for value in field_ranges:
        if value[0] <= number <= value[1] or value[2] <= number <= value[3]:
            number_valid = True
            # break
    return number_valid

error_rate = 0
nearby_tickets_valid = []
for ticket in nearby_tickets:
    ticket_valid = True
    for number in ticket:
        if not number_in_any_range(number):
            # one number on ticket is outside of every single range
            error_rate += number
            ticket_valid = False
    if ticket_valid:
        nearby_tickets_valid.append(ticket)

print('result 1: ', error_rate)

result 1:  20058


In [3]:
# ─── CHALLENGE 2 ────────────────────────────────────────────────────────────────

from math import prod

# rule id candidates for all 20 ticket id fields:
rule_id_candidates = {i: set() for i in range(20)}
solution = dict()

def number_fulfills_rule(number, rule_id):
    rule = field_ranges[rule_id]
    return rule[0] <= number <= rule[1] or rule[2] <= number <= rule[3]

# for every field and ticket count, how often the ticket_id -> rule_id relation
# is valid (is withing allowed range) 
for ticket_id in range(20):
    for rule_id in range(20):
        counter = 0
        for ticket in nearby_tickets_valid:
            number = ticket[ticket_id]
            if number_fulfills_rule(number, rule_id):
                counter += 1
        assert counter <= len(nearby_tickets_valid) # if not bug
        if counter == len(nearby_tickets_valid):
            rule_id_candidates[ticket_id].add(rule_id)

for _ in range(20):
    # assign solution, if only one possible solution is found
    for ticket_id, rule_ids in rule_id_candidates.items():
        if len(rule_ids) == 1:
            rule_id_found = rule_ids.pop()
            solution.update({rule_id_found: ticket_id})

    # removed assigned id from candidate sets
    for value in rule_id_candidates.values():
        value.discard(rule_id_found)

# multiply all ticket values with field name starting with departure (rules 0-5)
print('result 2: ', prod([your_ticket[solution[rule_id]] for rule_id in range(6)]))

result 2:  366871907221
